In [1]:
%gui qt5
%matplotlib qt5

In [11]:
import time
import numpy as np
import matplotlib as mpl

from importlib import reload
from matplotlib import pyplot as plt

from labtools import mplplots; reload(mplplots)
from labtools.mplplots.init_nb_plotting import *
from labtools.mplplots import tools as mpltools

import qcodes as qc
from qcodes.dataset.measurements import Measurement
from qcodes.instrument.parameter import ManualParameter
from qcodes.sweep import sweep
from qcodes.sweep.sweep import time_trace

from v2_dataset_tools.experiments import do_experiment, get_results_from_db_path

from qtpy.QtWidgets import QApplication
# from qtpy.QtGui import QPixmap

In [5]:
def update_plot_window(fig=None, activate_window=True):
    if fig is not None:
        fig.canvas.draw()
        if activate_window:
            fig.canvas.manager.window.activateWindow()
            fig.canvas.manager.window.raise_()
    QApplication.processEvents()

In [9]:
# the most minimal example.
# plot window updates fine, but interacting with the window blocks the measurement. that means it's unusable.
m = ManualParameter("m", unit="A")
m.get = lambda: np.sin((time.time() - t0) * 2 * np.pi / 2)

tvals = []
mvals = []

fig, ax = plt.subplots(1,1)

t0 = time.time()
for i in range(20):
    t = time.time() - t0
    tvals.append(t)
    mvals.append(m())
    
    ax.clear()
    ax.plot(tvals, mvals, 'ko')
    update_plot_window(fig)
    
    time.sleep(0.1)

In [26]:
# this 

x = ManualParameter('t', unit='s')
x.get = lambda: time.time() - t0

t0 = time.time()

meas = Measurement()
meas.register_parameter(x)
meas.register_parameter(m, setpoints=(x, ))

fig, ax = plt.subplots(1,1)
fig2, ax2 = plt.subplots(1,1)

with meas.run() as datasaver:
    tvals = []
    mvals = []
    for i in range(20):
        t = x()
        mval = m()
        tvals.append(t)
        mvals.append(mval)
        datasaver.add_result((x, t), (m, mval))
        
        ax.clear()
        ax.plot(tvals, mvals, 'ko')
        update_plot_window(fig)
        
        tvals2 = datasaver.dataset.get_data('t')
        mvals2 = datasaver.dataset.get_data('m')
        ax2.clear()
        ax2.plot(tvals2, mvals2, 'ro')
        update_plot_window(fig2)
        
        time.sleep(0.1)
        
tvals2 = datasaver.dataset.get_data('t')
mvals2 = datasaver.dataset.get_data('m')
ax2.clear()
ax2.plot(tvals2, mvals2, 'ro')
update_plot_window(fig2)

Starting experimental run with id: 42
